<a href="https://colab.research.google.com/github/kdubs100/CS_793/blob/main/CS_793_Assignment_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install numpy
!pip install transformers
!pip install scikit-learn


# 1st BERT model

In [ ]:
import numpy as np
import torch
from datasets import load_dataset
from transformers import AutoModel, AutoTokenizer

raw_data1 = load_dataset(
    'dell-research-harvard/headlines-semantic-similarity',
    data_files=["1929_headlines.json"]
)
#print(raw_data)

sample1 = np.random.choice(raw_data1['train'], 10000, replace=False)
sample1_headlines = []

for data_dict1 in sample1:
    if 'headline' in data_dict1:
        sample1_headlines.append(data_dict1['headline'])
#print(sample_headlines)


model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

#embeddings created in batches
embedded_headlines1 = []
batch_size = 100
i = 0
while i < len(sample1_headlines):
    batch_headlines = sample1_headlines[i:i+batch_size]
    max_seq_length = 16
    input_headlines = tokenizer(batch_headlines, return_tensors="pt", padding="max_length", truncation=True, max_length=max_seq_length)
    outputs_headlines = model(**input_headlines)
    embedded = outputs_headlines.last_hidden_state.mean(dim=1).squeeze().detach().numpy()
    embedded_headlines1.append(embedded)
    i += batch_size

embedded_headlines1 = np.concatenate(embedded_headlines1, axis=0)

#print(embedded_headlines)
print(len(embedded_headlines1))

10000


In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

range_n_clusters = [ 2, 3, 4, 5, 6, 7, 8]
averages1 = []
for n_clusters in range_n_clusters:

    clusterer = KMeans(n_clusters=n_clusters, init='random', n_init="auto", random_state=10)
    cluster_labels = clusterer.fit_predict(embedded_headlines1)

    sample_silhouette_values1 = silhouette_samples(embedded_headlines1, cluster_labels)

    silhouette_avg1 = silhouette_score(embedded_headlines1, cluster_labels)
    averages1.append((n_clusters, silhouette_avg1))
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg1,
    )

max_cluster1 = max(averages1, key=lambda x: x[1])
print(max_cluster1)

For n_clusters = 2 The average silhouette_score is : 0.054011058
For n_clusters = 3 The average silhouette_score is : 0.04838053
For n_clusters = 4 The average silhouette_score is : 0.03178407
For n_clusters = 5 The average silhouette_score is : 0.032822054
For n_clusters = 6 The average silhouette_score is : 0.03412041
For n_clusters = 7 The average silhouette_score is : 0.033235606
For n_clusters = 8 The average silhouette_score is : 0.032927938
(2, 0.054011058)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import random

cluster_idx_to_sample = (int(max_cluster1[0])-1)
clusterer = KMeans(n_clusters=cluster_idx_to_sample, init='random', n_init="auto", random_state=10)
cluster_idx_to_print = cluster_idx_to_sample


cluster_headlines1 = [sample1_headlines[i] for i, label in enumerate(cluster_labels) if label == cluster_idx_to_print]
#print(f"Cluster {cluster_idx_to_print + 1} Headlines:")
#for i, headline in enumerate(cluster_headlines1):
#    print(f"Headline {i + 1}: {headline}")

selected1 = random.sample(cluster_headlines1, 5)
#print(selected1)
selected_indices1 = [i for i, headline in enumerate(cluster_headlines1) if headline in selected1]
#print(selected_indices1)
selected_headlines1 = [cluster_headlines1[i] for i in selected_indices1]

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(cluster_headlines1 + selected1)

cosine_similarities1 = cosine_similarity(tfidf_matrix[-6:-1], tfidf_matrix[:-len(selected1)])
#print(cosine_similarities1)
for i, selected_headlines1 in enumerate(selected1):
    print(f"Selected Headline {i + 1}: {selected_headlines1}")
    top_indices = cosine_similarities1[i].argsort()[-5:][::-1]
    print(top_indices)
    print("Top 5 Closest Headlines:")
    for j, index in enumerate(top_indices):
        print(f"  Similar Headline {j + 1}: {cluster_headlines1[index]} (Cosine Similarity: {cosine_similarities1[i][index]})")



Selected Headline 1: AMERICA  BIVEN PHAloE   Citizens of U. S. Are Understanding Us, MacDonald Says   NATION SYMPATHETIC
[3670 1606 1156   91 2039]
Top 5 Closest Headlines:
  Similar Headline 1: FREED AS COURT DECIDES POLICE ARE PSCONSESTENS (Cosine Similarity: 1.0000000000000002)
  Similar Headline 2: Thomas W. Miller  Freed From “Pen’ (Cosine Similarity: 0.19209971221453814)
  Similar Headline 3: I’m Alone Captain And Crew Freed Of Charge: (Cosine Similarity: 0.18230064745371538)
  Similar Headline 4: hLublicates Police (Cosine Similarity: 0.16083218485559977)
  Similar Headline 5: Robbers Held For Court (Cosine Similarity: 0.15784049424567234)
Selected Headline 2: Somerset Conspirators Post Bond for Court
[1155 1925  295  802 1610]
Top 5 Closest Headlines:
  Similar Headline 1: AMERICA  BIVEN PHAloE   Citizens of U. S. Are Understanding Us, MacDonald Says   NATION SYMPATHETIC (Cosine Similarity: 0.9999999999999998)
  Similar Headline 2: Cause Of Peace Unites : Us, Says MacDonald In 

In [ ]:
#print(selected1)

similar1 = cosine_similarity(tfidf_matrix[-len(selected1):], tfidf_matrix[:-len(selected1)])
selected_sim1 = []
#print(similar1)
for i, selected_headlines1 in enumerate(selected1):
  new_indices1 = similar1[i].argsort()[-10:-8][::-1]
#  print(new_indices1)
  selected_headlines_set = set()  # Initialize a set for the selected headlines
  selected_cos = []
  for index in new_indices1:
      selected_headlines_set.add(cluster_headlines1[index])
      selected_cos.append(cosine_similarities1[i][index])

  selected_sim1.append((selected_headlines_set,selected_cos))

for j, similar1_headline in enumerate(selected1):
    print(f"Selected Headline {j + 1}: {similar1_headline}")
    selected_headlines_set, selected_cos = selected_sim1[j]
    for i, selected_headline in enumerate(selected_headlines_set):
        print(f"Similar Headline: {selected_headline}\n Cosine Similarity {selected_cos[i]}")


Selected Headline 1: AMERICA  BIVEN PHAloE   Citizens of U. S. Are Understanding Us, MacDonald Says   NATION SYMPATHETIC
Similar Headline: Premier MacDonald ls On Way Back Home
 Cosine Similarity 0.0
Similar Headline: Asks Nation To Stand Ready For  Soviet ‘Menace
 Cosine Similarity 0.0
Selected Headline 2: Somerset Conspirators Post Bond for Court
Similar Headline: NEBRASKA EDITOR WILL TAKE FARM BOARD POST
 Cosine Similarity 0.015906454897169513
Similar Headline: Ministers Held for Contempt of Court
 Cosine Similarity 0.0
Selected Headline 3: Borah Asks Britain to Sink Some of Her Ship:
Similar Headline: PEACOX. CONFESSES. TO KILLING AND THEN BIBAING AE HIS WIFE   Said That He Hit Her on the Head With Pistol and Then Straneled Her.   POURED KEROSENE   On Her Body and Clothing And Set Fire to It— * Girl is Held.
 Cosine Similarity 0.0
Similar Headline: President Hoover Responds to British Ship Curtailment   AMERICA WILL ADOPT . PRINCIPLE OF PARITY   Great Britain and United States ta C

In [ ]:
differ1 = []
diff_indices = []
for i in range(0,5):
    rand = random.sample(cluster_headlines1, 1)
    differ1.append(rand)
    ind = [idx for idx, headline in enumerate(cluster_headlines1) if headline == rand[0]]
    diff_indices.append(ind)

diff1 = []
for indices in diff_indices:
    for index in indices:
        headline = cluster_headlines1[index]
        cos_sim = cosine_similarities1[i][index]
        diff1.append((headline, cos_sim))

for j, similar1_headline in enumerate(selected1):
    print(f"Selected Headline {j + 1}: {similar1_headline}")
    headline, cos_sim = diff1[j]
    print(f"Headline: {headline} Cosine Similarity: {cos_sim}\n")

#print(diff1)
#print(differ1)
#print(diff_indices)


Selected Headline 1: AMERICA  BIVEN PHAloE   Citizens of U. S. Are Understanding Us, MacDonald Says   NATION SYMPATHETIC
Headline: BOTH HOUSES ARE . ON FARM JOB NOW Cosine Similarity: 0.0

Selected Headline 2: Somerset Conspirators Post Bond for Court
Headline: “6 FIREMEN ARE TRAPPED BY WALL; i CRUSHED TO DEATH Cosine Similarity: 0.0

Selected Headline 3: Borah Asks Britain to Sink Some of Her Ship:
Headline: FOUR HUNDRED SKATERS GET DIP AS ICE BREAKS Cosine Similarity: 0.0

Selected Headline 4: ST. MARY’S CO. REGAINS PERMIT
Headline: THIRTY THREE HURT IN TRAIN WRECK TODAY   Burlington Passenger Trair Backs into Another at St. Louis Train Shed.   TWO COACHES FILLED _ WITH PEOPLE CRUSHED   All Victims Expected to Re. cover; Officials Start Probe of Cause. Cosine Similarity: 0.05916077891773062

Selected Headline 5: BLOLWILED LUMO CAUSE OF GUNPLAY T0 “BREAK SPELL”   SANITY HEARING ASKED FOR STURGEON BAY COUNTRYMAN  NEIGHBOR TARGET   Harmless Fusillade Results in Jail Bunk for Superstitio

The results of the cosine similarities indicate that even "closer" headlines are not necessarily considered close. I think for this model, it is likely because of the cluster size. I had to reduce the sequence length of the embeddings to reduce the amount of RAM used which resulted in ill fitting clusters for this model. However, the "similar" headlines still resulted in much better results than the random headlines. Additionally, some of the variation would be due to the model used to embed the headlines. The second BERT type model I used was larger and resulted in results with higher cosine similaries. In the third model, the clusters were extremely poor fitting resulting in high cosine similarities for headlines that were not similar in context.

# 2nd BERT model


In [ ]:
raw_data2 = load_dataset(
    'dell-research-harvard/headlines-semantic-similarity',
    data_files=["1929_headlines.json"]
)
#print(raw_data)

sample2 = np.random.choice(raw_data2['train'], 10000, replace=False)
sample2_headlines = []

for data_dict2 in sample2:
    if 'headline' in data_dict2:
        sample2_headlines.append(data_dict2['headline'])
#print(sample_headlines)


model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

#embeddings created in batches
embedded_headlines2 = []
batch_size = 100
i = 0
while i < len(sample2_headlines):
    batch_headlines = sample2_headlines[i:i+batch_size]
    max_seq_length = 16
    input_headlines = tokenizer(batch_headlines, return_tensors="pt", padding="max_length", truncation=True, max_length=max_seq_length)
    outputs_headlines = model(**input_headlines)
    embedded = outputs_headlines.last_hidden_state.mean(dim=1).squeeze().detach().numpy()
    embedded_headlines2.append(embedded)
    i += batch_size

embedded_headlines2 = np.concatenate(embedded_headlines2, axis=0)

#print(embedded_headlines)
print(len(embedded_headlines2))

10000


In [ ]:
range_n_clusters = [ 2, 3, 4, 5, 6, 7, 8]
averages2 = []
for n_clusters in range_n_clusters:

    # Initialize the clusterer with n_clusters value and other parameters as needed
    clusterer = KMeans(n_clusters=n_clusters, init='random', n_init="auto", random_state=10)
    cluster_labels = clusterer.fit_predict(embedded_headlines2)

    # Compute the silhouette scores for each sample
    sample_silhouette_values2 = silhouette_samples(embedded_headlines2, cluster_labels)

    # Calculate the average silhouette score for the clustering
    silhouette_avg2 = silhouette_score(embedded_headlines2, cluster_labels)
    averages2.append((n_clusters, silhouette_avg2))
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg2,
    )

max_cluster2 = max(averages2, key=lambda x: x[1])
print(max_cluster2)

For n_clusters = 2 The average silhouette_score is : 0.307241
For n_clusters = 3 The average silhouette_score is : 0.09764994
For n_clusters = 4 The average silhouette_score is : 0.055352516
For n_clusters = 5 The average silhouette_score is : 0.040002003
For n_clusters = 6 The average silhouette_score is : 0.037365384
For n_clusters = 7 The average silhouette_score is : 0.030984377
For n_clusters = 8 The average silhouette_score is : 0.030426977
(2, 0.307241)


In [ ]:
cluster_idx_to_sample = (int(max_cluster2[0])-1)
clusterer = KMeans(n_clusters=cluster_idx_to_sample, init='random', n_init="auto", random_state=10)
cluster_idx_to_print = cluster_idx_to_sample


cluster_headlines2 = [sample2_headlines[i] for i, label in enumerate(cluster_labels) if label == cluster_idx_to_print]
#print(f"Cluster {cluster_idx_to_print + 1} Headlines:")
#for i, headline in enumerate(cluster_headlines1):
#    print(f"Headline {i + 1}: {headline}")

selected2 = random.sample(cluster_headlines2, 5)
#print(selected1)
selected_indices2 = [i for i, headline in enumerate(cluster_headlines2) if headline in selected2]
#print(selected_indices1)
selected_headlines2 = [cluster_headlines2[i] for i in selected_indices2]

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(cluster_headlines2 + selected2)

cosine_similarities2 = cosine_similarity(tfidf_matrix[-6:-1], tfidf_matrix[:-len(selected2)])
#print(cosine_similarities1)
for i, selected_headlines2 in enumerate(selected2):
    print(f"Selected Headline {i + 1}: {selected_headlines2}")

    # Get the indices of the 5 headlines with the highest cosine similarity scores
    top_indices = cosine_similarities2[i].argsort()[-5:][::-1]

    print("Top 5 Closest Headlines:")
    for j, index in enumerate(top_indices):
        print(f"  Similar Headline {j + 1}: {cluster_headlines2[index]} (Cosine Similarity: {cosine_similarities2[i][index]})")



Selected Headline 1: ‘Collapse of Trial to Be Investigated in Chicago
Top 5 Closest Headlines:
  Similar Headline 1: White Squaws Make Thunder Over Nothing   Chief Two Guns Offer Solution Of Capital’s  Social Fuss (Cosine Similarity: 1.0000000000000002)
  Similar Headline 2: Negroes Murder White in Quarrel Over Right-of-Way (Cosine Similarity: 0.14408007642830406)
  Similar Headline 3: Refused $50,000 Dog Track Offer (Cosine Similarity: 0.12900134774532024)
  Similar Headline 4: Negro Shot Down After Shooting Two White Mer (Cosine Similarity: 0.12658055251943312)
  Similar Headline 5: New German Ship. Has 11-Inch Guns’ (Cosine Similarity: 0.1168854262965212)
Selected Headline 2: Weather
Top 5 Closest Headlines:
  Similar Headline 1: ‘Collapse of Trial to Be Investigated in Chicago (Cosine Similarity: 1.0000000000000002)
  Similar Headline 2: Chicago’s Gangland to Be Investigated by Crime Commissior (Cosine Similarity: 0.43611672502539733)
  Similar Headline 3: Cotton Exchanges to Be In

In [ ]:
#print(selected2)

similar2 = cosine_similarity(tfidf_matrix[-len(selected2):], tfidf_matrix[:-len(selected2)])
selected_sim2 = []
#print(similar2)
for i, selected_headlines2 in enumerate(selected2):
  new_indices2 = similar2[i].argsort()[-10:-8][::-1]
#  print(new_indices2)
  selected_headlines_set = set()  # Initialize a set for the selected headlines
  selected_cos = []
  for index in new_indices2:
      selected_headlines_set.add(cluster_headlines2[index])
      selected_cos.append(cosine_similarities2[i][index])

  selected_sim2.append((selected_headlines_set, selected_cos))

for j, similar2_headline in enumerate(selected2):
    print(f"Selected Headline {j + 1}: {similar2_headline}")
    selected_headlines_set, selected_cos = selected_sim2[j]
    for i, selected_headline in enumerate(selected_headlines_set):
        print(f"Similar Headline: {selected_headline}\n Cosine Similarity {selected_cos[i]}")


Selected Headline 1: ‘Collapse of Trial to Be Investigated in Chicago
Similar Headline: Open Trial Of Arlie Boswel
 Cosine Similarity 0.0
Similar Headline: ‘Heflin Fails to Annear for Trial
 Cosine Similarity 0.015089878241981868
Selected Headline 2: Weather
Similar Headline: Cooler Weather Finally Arrives
 Cosine Similarity 0.0
Similar Headline: Endurance Plane Is Defeated By Weather
 Cosine Similarity 0.0
Selected Headline 3: U. S. Owns Turtle Islands, Britain Governs Them   Treaty Now Being Completed That Will Settle Sovereignty Of Important Little Group Of Philip pines.
Similar Headline: Embargo Treaty Ratified.
 Cosine Similarity 0.0
Similar Headline: Britain Slows New Navy Work
 Cosine Similarity 0.0
Selected Headline 4: a ‘Today  The President  > — Investigates  ; Bles: Terror Ab: Arabs  > Der A wthiir RPereichana
Similar Headline: Ex- President’s Son Will Marry Daughter Of Connecticut’s Governor Late Today
 Cosine Similarity 0.0442651915506696
Similar Headline: Will Rogers Says

In [ ]:
differ2 = []
diff_indices2 = []
for i in range(0, 5):
    rand = random.sample(cluster_headlines2, 1)
    differ2.append(rand)
    ind = [idx for idx, headline in enumerate(cluster_headlines2) if headline == rand[0]]
    diff_indices2.append(ind)

diff2 = []
for indices in diff_indices2:
    for index in indices:
        headline = cluster_headlines2[index]
        cos_sim = cosine_similarities2[i][index]
        diff2.append((headline, cos_sim))

for j, similar2_headline in enumerate(selected2):
    print(f"Selected Headline {j + 1}: {similar2_headline}")
    headline, cos_sim = diff2[j]
    print(f"Headline: {headline} Cosine Similarity: {cos_sim}\n")

#print(diff2)
#print(differ2)
#print(diff_indices2)


Selected Headline 1: ‘Collapse of Trial to Be Investigated in Chicago
Headline: Missionary Basket Meeting Friday  P.-M.~“Annual ' Evans: Barbecuc Today—Social. and- Personal. © Cosine Similarity: 0.041672987959726286

Selected Headline 2: Weather
Headline: Governor Hammill Helps in Dedication of Missouri Bridge Cosine Similarity: 0.0

Selected Headline 3: U. S. Owns Turtle Islands, Britain Governs Them   Treaty Now Being Completed That Will Settle Sovereignty Of Important Little Group Of Philip pines.
Headline: Memorial Services Helc Sunday at Spurgeon and First M. E. Churches Cosine Similarity: 0.0

Selected Headline 4: a ‘Today  The President  > — Investigates  ; Bles: Terror Ab: Arabs  > Der A wthiir RPereichana
Headline: Wholesale - Bootlegger Is Blamed for Seventeen — Deaths at Peoria Cosine Similarity: 0.0

Selected Headline 5: Lindbergh. Honeymoon Yacht Resumes Voyage
Headline: 17 Drown Cosine Similarity: 0.0



# 3rd BERT Model

In [ ]:
raw_data3 = load_dataset(
    'dell-research-harvard/headlines-semantic-similarity',
    data_files=["1929_headlines.json"]
)
#print(raw_data)

sample3 = np.random.choice(raw_data3['train'], 10000, replace=False)
sample3_headlines = []

for data_dict3 in sample3:
    if 'headline' in data_dict3:
        sample3_headlines.append(data_dict3['headline'])
#print(sample_headlines)


model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

#embeddings created in batches
embedded_headlines3 = []
batch_size = 100
i = 0
while i < len(sample3_headlines):
    batch_headlines = sample3_headlines[i:i+batch_size]
    max_seq_length = 16
    input_headlines = tokenizer(batch_headlines, return_tensors="pt", padding="max_length", truncation=True, max_length=max_seq_length)
    outputs_headlines = model(**input_headlines)
    embedded = outputs_headlines.last_hidden_state.mean(dim=1).squeeze().detach().numpy()
    embedded_headlines3.append(embedded)
    i += batch_size

embedded_headlines3 = np.concatenate(embedded_headlines3, axis=0)

#print(embedded_headlines)
print(len(embedded_headlines3))

10000


In [ ]:
range_n_clusters = [ 2, 3, 4, 5, 6, 7, 8]
averages3 = []
for n_clusters in range_n_clusters:

    # Initialize the clusterer with n_clusters value and other parameters as needed
    clusterer = KMeans(n_clusters=n_clusters, init='random', n_init="auto", random_state=10)
    cluster_labels = clusterer.fit_predict(embedded_headlines2)

    # Compute the silhouette scores for each sample
    sample_silhouette_values3 = silhouette_samples(embedded_headlines3, cluster_labels)

    # Calculate the average silhouette score for the clustering
    silhouette_avg3 = silhouette_score(embedded_headlines3, cluster_labels)
    averages3.append((n_clusters, silhouette_avg2))
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg3,
    )

max_cluster3 = max(averages3, key=lambda x: x[1])
print(max_cluster3)

For n_clusters = 2 The average silhouette_score is : 0.00039262153
For n_clusters = 3 The average silhouette_score is : -0.0017729021
For n_clusters = 4 The average silhouette_score is : -0.0026789864
For n_clusters = 5 The average silhouette_score is : -0.0049171164
For n_clusters = 6 The average silhouette_score is : -0.008298793
For n_clusters = 7 The average silhouette_score is : -0.010409976
For n_clusters = 8 The average silhouette_score is : -0.011564627
(2, 0.030426977)


In [ ]:
cluster_idx_to_sample = (int(max_cluster3[0])-1)
clusterer = KMeans(n_clusters=cluster_idx_to_sample, init='random', n_init="auto", random_state=10)
cluster_idx_to_print = cluster_idx_to_sample


cluster_headlines3 = [sample3_headlines[i] for i, label in enumerate(cluster_labels) if label == cluster_idx_to_print]
#print(f"Cluster {cluster_idx_to_print + 1} Headlines:")
#for i, headline in enumerate(cluster_headlines1):
#    print(f"Headline {i + 1}: {headline}")

selected3 = random.sample(cluster_headlines3, 5)
#print(selected1)
selected_indices3 = [i for i, headline in enumerate(cluster_headlines3) if headline in selected3]
#print(selected_indices1)
selected_headlines3 = [cluster_headlines3[i] for i in selected_indices3]

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(cluster_headlines3 + selected3)

cosine_similarities3 = cosine_similarity(tfidf_matrix[-6:-1], tfidf_matrix[:-len(selected3)])
#print(cosine_similarities1)
for i, selected_headlines3 in enumerate(selected3):
    print(f"Selected Headline {i + 1}: {selected_headlines3}")

    # Get the indices of the 5 headlines with the highest cosine similarity scores
    top_indices = cosine_similarities3[i].argsort()[-5:][::-1]

    print("Top 5 Closest Headlines:")
    for j, index in enumerate(top_indices):
        print(f"  Similar Headline {j + 1}: {cluster_headlines3[index]} (Cosine Similarity: {cosine_similarities3[i][index]})")



Selected Headline 1: EXTRA JUROR IS PLACED IN BOX: PICK ANOTHER   Melvin Hix, Father of Slain Girl, Also Ill and Not in Court   SNOOK VERY WEAK   Defendant is Pleased at  Delay in His Trial; Rest is Welcome .
Top 5 Closest Headlines:
  Similar Headline 1: Hunt Slayer (Cosine Similarity: 1.0)
  Similar Headline 2: Richmond Broker Killed On Hunt (Cosine Similarity: 0.37938331741745596)
  Similar Headline 3: Wife’s Slayer Will Get Her Insurance (Cosine Similarity: 0.27947771429655693)
  Similar Headline 4: FRENCH JURY -ACQUITS SLAYER   Man Who Murdered Invalic Mother to Stop Suffering ‘Not to Be Punished. (Cosine Similarity: 0.1717087968594731)
  Similar Headline 5: BOY 1S LOCATED AFTER LONG HUNT   His Face In Smiles, His Cloth ing In Tatters, Lad Coast: On Roller Skates Into Arm Of Officers (Cosine Similarity: 0.1670814857501556)
Selected Headline 2: Rerencaria Steame On
Top 5 Closest Headlines:
  Similar Headline 1: EXTRA JUROR IS PLACED IN BOX: PICK ANOTHER   Melvin Hix, Father of Slai

In [ ]:
#print(selected3)

similar3 = cosine_similarity(tfidf_matrix[-len(selected3):], tfidf_matrix[:-len(selected3)])
selected_sim3 = []
#print(similar3)
for i, selected_headlines3 in enumerate(selected3):
  new_indices3 = similar3[i].argsort()[-10:-8][::-1]
#  print(new_indices3)
  selected_headlines_set = set()  # Initialize a set for the selected headlines
  selected_cos = []
  for index in new_indices3:
      selected_headlines_set.add(cluster_headlines3[index])
      selected_cos.append(cosine_similarities3[i][index])

  selected_sim3.append((selected_headlines_set, selected_cos))

for j, similar3_headline in enumerate(selected3):
    print(f"Selected Headline {j + 1}: {similar3_headline}")
    selected_headlines_set, selected_cos = selected_sim3[j]
    for i, selected_headline in enumerate(selected_headlines_set):
        print(f"Similar Headline: {selected_headline}\n Cosine Similarity {selected_cos[i]}")


Selected Headline 1: EXTRA JUROR IS PLACED IN BOX: PICK ANOTHER   Melvin Hix, Father of Slain Girl, Also Ill and Not in Court   SNOOK VERY WEAK   Defendant is Pleased at  Delay in His Trial; Rest is Welcome .
Similar Headline: Duties of a New Father-in-Law Not Easy, Envoy Finds
 Cosine Similarity 0.0
Similar Headline: Catholic Priest Is Slain by Bandits’ in China, Mission Hear:
 Cosine Similarity 0.0
Selected Headline 2: Rerencaria Steame On
Similar Headline: Irge Work on Hichway 7%
 Cosine Similarity 0.0
Similar Headline: Hoover Continues  On Fishing Trip
 Cosine Similarity 0.0
Selected Headline 3: Zeppelin, Byrd Flights and - Wall Street Crash Rated Leading 1929 News Events
Similar Headline:  Victims of Crash  Were All Praminen!
 Cosine Similarity 0.0
Similar Headline: Easter Greetings Are _ Broadcast to Public ~ From Byrd in Antarctic
 Cosine Similarity 0.0
Selected Headline 4: IOWA CITY GIRL KILLS HERSEL!      Miss Hollis Foote Shot Her. self Through the Heart Early Today
Similar H

In [ ]:
differ3 = []
diff_indices3 = []
for i in range(0, 5):
    rand = random.sample(cluster_headlines3, 1)
    differ3.append(rand)
    ind = [idx for idx, headline in enumerate(cluster_headlines3) if headline == rand[0]]
    diff_indices3.append(ind)

diff3 = []
for indices in diff_indices3:
    for index in indices:
        headline = cluster_headlines3[index]
        cos_sim = cosine_similarities3[i][index]
        diff3.append((headline, cos_sim))

for j, similar3_headline in enumerate(selected3):
    print(f"Selected Headline {j + 1}: {similar3_headline}")
    headline, cos_sim = diff3[j]
    print(f"Headline: {headline} Cosine Similarity: {cos_sim}\n")

#print(diff3)
#print(differ3)
#print(diff_indices3)


Selected Headline 1: EXTRA JUROR IS PLACED IN BOX: PICK ANOTHER   Melvin Hix, Father of Slain Girl, Also Ill and Not in Court   SNOOK VERY WEAK   Defendant is Pleased at  Delay in His Trial; Rest is Welcome .
Headline: DEATH OF IOWA STATE SENATOR UNDER INQUIR’   George: ‘Ly. Finn, Often Ex E Cosine Similarity: 0.08184661348136706

Selected Headline 2: Rerencaria Steame On
Headline: 3006 UNABLE TO BEAT DOWN FIRE   Forest Flames Lap Up Rich Timber Cosine Similarity: 0.0

Selected Headline 3: Zeppelin, Byrd Flights and - Wall Street Crash Rated Leading 1929 News Events
Headline: House Puts Dry Bill in Conterence   ‘Party Lines Shattered ix  Tilt Over $24,000,000 Measure. Cosine Similarity: 0.0

Selected Headline 4: IOWA CITY GIRL KILLS HERSEL!      Miss Hollis Foote Shot Her. self Through the Heart Early Today
Headline: SOCIAL LEADERS GATHER FOR CIVIC OPERA WINTER OPENING ATNEW THEATER Cosine Similarity: 0.0

Selected Headline 5: tEPARATIONS CONFERENCE ENDS LABORS WITH APPOINTMENT OF SUB-